In [1]:
# ! pip install -r requirements.txt

In [1]:
API_URL="http://116.202.111.229:8000"
API_KEY="Ir7nzlSO3ZBEdGV59Ll0FMVtPsNd85Cz"
DATA_PATH = '/root/HBN/Tournament/DS/'
CACHE_DIR = '/workspace/octavf'

# TOURNAMENT

In [2]:
from evaluator import Evaluator
from api_client import APIClient
import time

ev = Evaluator(api_url=API_URL, api_key=API_KEY, api_client=APIClient(API_URL, API_KEY),
               DATA_PATH=DATA_PATH, cache_dir=CACHE_DIR)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:

# ev.api_client.get_next_hint_for_current_company()
# ev.api_client.reset_current_context()

In [3]:
# ev.evaluate(sleep=True)

In [6]:
while True:
    try:
        for i in range(5):
            ev.compete(round_id=i+1, sleep=True)
    except Exception as ex:
        print(ex)
    time.sleep(200)

200 {'company_id': 5, 'level': 1, 'hint': 'Heart to Heart'}


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answered {'answer': 'Health and Personal Care Retailers'}
Verdict 200 {'result': 'wrong', 'score': -50.0, 'answer': '624 Social Assistance'}
200 {'company_id': 5, 'level': 2, 'hint': 'Global Health Information Provider | Humanitarian Services | Disaster Management Services | Direct Patient Care | Medication Distribution | Hygiene Supplies Distribution | Healthcare Access Improvement | Domestic Health Services'}


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answered {'answer': 'Web Search Portals, Libraries, Archives, and Other Information Services'}
Verdict 200 {'result': 'wrong', 'score': -100.0, 'answer': '624 Social Assistance'}
200 {'company_id': 5, 'level': 3, 'hint': 'As a healthcare nonprofit organization, Heart to Heart International invests in Kansas City to create a healthier world.'}


...

Answered {'answer': 'Ambulatory Health Care Services'}
Verdict 200 {'result': 'wrong', 'score': -150.0, 'answer': '624 Social Assistance'}
200 {'company_id': 5, 'level': 4, 'hint': 'Heart to Heart International is a non-profit organization that aims to improve healthcare access in the United States and around the world by ensuring quality care is provided equitably in medically under-resourced communities and disaster situations. They have provided humanitarian service in over 130 countries, shipped over $2.4 billion in aid, and logged more than 1 million volunteer hours. The organization focuses on improving access to health, community health, laboratory programs, and global health. They offer various ways to get involved, including donating medical supplies, donating hygiene kits, and volunteering. Heart to Heart has a history of success in improving healthcare access and has shipped products to various countries.'}


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answered {'answer': 'Ambulatory Health Care Services'}
Verdict 200 {'result': 'wrong', 'score': -200.0, 'answer': '624 Social Assistance'}
200 {'company_id': 5, 'level': 5, 'hint': 'Social Services & NGOs'}


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answered {'answer': 'Ambulatory Health Care Services'}
Verdict 200 {'result': 'wrong', 'score': -250.0, 'answer': '624 Social Assistance'}
200 {'response': 'Wait for 273.09413480758667 seconds'}


# CHALLENGE

In [3]:
from classifier import NAICSClassifier, TokenizerWrapper
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
import numpy as np
from collections import defaultdict

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1", device_map="auto", load_in_4bit=True, cache_dir=CACHE_DIR)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", padding_side="left", cache_dir=CACHE_DIR)
tokenizer = TokenizerWrapper(tokenizer, model)
n2 = NAICSClassifier(tokenizer=tokenizer, naics_df=f'/{DATA_PATH}/ml_tournament/resources/NAICS2_324254.csv')
naics2_df = pd.read_csv(f"{DATA_PATH}/ml_tournament/resources/NAICS2_324254.csv")
naics4_df = pd.read_csv(f"{DATA_PATH}/ml_tournament/resources/NAICS4_324254.csv")
naics6_df = pd.read_csv(f"{DATA_PATH}/ml_tournament/resources/NAICS6_324254.csv")

company_hints = []
response = defaultdict(str)

n2_classes_scores = n2.classify_list_of_hints(company_hints)
naics_code = naics2_df[naics2_df['naics_label'] == n2.naics3_choices[np.argmax(n2_classes_scores)]]['naics_code']
response['naics2'] = naics_code

# based on the result, n4:
n4 = NAICSClassifier(tokenizer=tokenizer, 
                     naics_df=naics4_df[naics4_df['naics_code'].apply(lambda subnsc: subnsc.startswith(naics_code))])
n4_classes_scores = n4.classify_list_of_hints(company_hints)
naics_code = naics4_df[naics4_df['naics_label'] == n4.naics3_choices[np.argmax(n4_classes_scores)]]['naics_code']
response['naics4'] = naics_code

# based on the result, n6:
n6 = NAICSClassifier(tokenizer=tokenizer, 
                     naics_df=naics6_df[naics6_df['naics_code'].apply(lambda subnsc: subnsc.startswith(naics_code))])

n6_classes_scores = n6.classify_list_of_hints(company_hints)
naics_code = naics6_df[naics6_df['naics_label'] == n6.naics3_choices[np.argmax(n6_classes_scores)]]['naics_code']
response['naics6'] = naics_code

print(response)


NameError: name 'CACHE_DIR' is not defined